**By 加号**

微博：[@翻滚吧\_加号](https://www.weibo.com/4u2go)

这一波的代码其实非常的简单,

就是给大家展示一下rule-based的玩法,

以及几个角度的升级。

首先，我们看一个

### 最基础版本的rule-base机器人

基本就是小学生级别的 问什么 答什么

In [1]:
import random

# 打招呼
greetings = ['hola', 'hello', 'hi', 'Hi', 'hey!','hey']
# 回复打招呼
random_greeting = random.choice(greetings)

# 对于“你怎么样？”这个问题的回复
question = ['How are you?','How are you doing?']
# “我很好”
responses = ['Okay',"I'm fine"]
# 随机选一个回
random_response = random.choice(responses)

# 机器人跑起来
while True:
    userInput = input(">>> ")
    if userInput in greetings:
        print(random_greeting)
    elif userInput in question:
        print(random_response)
    # 除非你说“拜拜”
    elif userInput == 'bye':
        break
    else:
        print("I did not understand what you said")

>>> hi
hey
>>> how are you
I did not understand what you said
>>> How are you?
Okay
>>> bye


### 升级I:

显然 这样的rule太弱智了，我们需要更好一点的“精准对答”

比如

透过关键词来判断这句话的意图是什么（intents）

In [2]:
from nltk import word_tokenize
import random

# 打招呼
greetings = ['hola', 'hello', 'hi', 'Hi', 'hey!','hey']
# 回复打招呼
random_greeting = random.choice(greetings)

# 对于“假期”的话题关键词
question = ['break','holiday','vacation','weekend']
# 回复假期话题
responses = ['It was nice! I went to Paris',"Sadly, I just stayed at home"]
# 随机选一个回
random_response = random.choice(responses)



# 机器人跑起来
while True:
    userInput = input(">>> ")
    # 清理一下输入，看看都有哪些词
    cleaned_input = word_tokenize(userInput)
    # 这里，我们比较一下关键词，确定他属于哪个问题
    if  not set(cleaned_input).isdisjoint(greetings):
        print(random_greeting)
    elif not set(cleaned_input).isdisjoint(question):
        print(random_response)
    # 除非你说“拜拜”
    elif userInput == 'bye':
        break
    else:
        print("I did not understand what you said")

>>> how was your holiday?
Sadly, I just stayed at home
>>> what did you do last weekend?
Sadly, I just stayed at home
>>> what's your plan for next vacation?
Sadly, I just stayed at home
>>> bye


大家大概能发现，这依旧是文字层面的“精准对应”。

现在主流的研究方向，是做到语义层面的对应。

比如，“肚子好饿哦”， “饭点到了”

都应该表示的是要吃饭了的意思。

在这个层面，就需要用到word vector之类的embedding方法，

这部分内容 日后的课上会涉及到。

### 升级II：

光是会BB还是不行，得有知识体系！才能解决用户的问题。

我们可以用各种数据库，建立起一套体系，然后通过搜索的方式，来查找答案。

比如，最简单的就是Python自己的graph数据结构来搭建一个“地图”。

依据这个地图，我们可以清楚的找寻从一个地方到另一个地方的路径，

然后作为回答，反馈给用户。

In [3]:

# 建立一个基于目标行业的database
# 比如 这里我们用python自带的graph
graph = {'上海': ['苏州', '常州'],
         '苏州': ['常州', '镇江'],
         '常州': ['镇江'],
         '镇江': ['常州'],
         '盐城': ['南通'],
         '南通': ['常州']}

# 明确如何找到从A到B的路径
def find_path(start, end, path=[]):
    path = path + [start]
    if start == end:
        return path
    if start not in graph:
        return None
    for node in graph[start]:
        if node not in path:
            newpath = find_path(node, end, path)
            if newpath: return newpath
    return None

In [4]:
print(find_path('上海', "镇江"))

['上海', '苏州', '常州', '镇江']


同样的构建知识图谱的玩法，

也可以使用一些Logic Programming，比如上个世纪学AI的同学都会学的Prolog。

或者比如，python版本的prolog：PyKE。

他们可以构建一种复杂的逻辑网络，让你方便提取信息，

而不至于需要你亲手code所有的信息:


    son_of(bruce, thomas, norma)
    son_of(fred_a, thomas, norma)
    son_of(tim, thomas, norma)
    daughter_of(vicki, thomas, norma)
    daughter_of(jill, thomas, norma)



### 升级III：

任何行业，都分个前端后端。

AI也不例外。

我们这里讲的算法，都是后端跑的。

那么， 为了做一个靠谱的前端，很多项目往往也需要一个简单易用，靠谱的前端。

比如，这里，利用Google的API，写一个类似钢铁侠Tony的语音小秘书Jarvis：

我们先来看一个最简单的说话版本。

利用gTTs(Google Text-to-Speech API), 把文本转化为音频。

In [2]:
#可以忽略，我只是为了页面美观
import warnings
warnings.simplefilter('ignore')

from gtts import gTTS
import os
tts = gTTS(text='您好，我是您的私人助手，我叫小辣椒', lang='zh-tw')
tts.save("hello.mp3")
os.system("mpg321 hello.mp3")

0

同理，

有了文本到语音的功能，

我们还可以运用Google API读出Jarvis的回复：

（注意：这里需要你的机器安装几个库 SpeechRecognition, PyAudio 和 PySpeech）

In [ ]:
import speech_recognition as sr
from time import ctime
import time
import os
from gtts import gTTS
import sys
from pypinyin import lazy_pinyin
 
# 讲出来AI的话
def speak(audioString):
    print(audioString)
    tts = gTTS(text=audioString, lang='zh-tw')
    tts.save("audio.mp3")
    os.system("mpg321 audio.mp3")

# 录下来你讲的话
def recordAudio():
    # 用麦克风记录下你的话
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r.listen(source)
 
    # 用Google API转化音频
    data = ""
    try:
        # 这里注意，我讲不出台湾腔，为了准确度，使用标准普通话
        data = r.recognize_google(audio, language='zh-cn')
        print("你说：" + data)
    except sr.UnknownValueError:
        #print("对不起，我不能理解你的话")
        #为了输出不要太多乱七八糟的东西，我这里直接pass
        pass
    except sr.RequestError as e:
        print("无法从谷歌服务中找到符合你要求的内容; {0}".format(e))
 
    return data

# 自带的对话技能（rules）
def jarvis():
    
    while True:
        
        data = recordAudio()
        
        #不让它什么都回，只回『小辣椒』开头的句子
        if "辣椒" in data:
            speak("我在！")
            #再听一句
            data = recordAudio()
            #问候
            if "你好吗" in data:
                speak("我很好")
            #时间
            elif "时间" in data:
                speak(ctime())
            #打开网页
            elif "在哪里" in data:
                #把后面三位：『在哪里』去掉
                location = data[:-3]
                speak("别着急宝贝，我给你看看" + location + "在哪里")
                #这里有个问题就是，google的url打开不了『上海』这个中文。如果换成pinyin的话就没问题了
                #所以我们用PyPinYin
                pinyin = lazy_pinyin(location)
                pinyin_location = ''
                for i in pinyin:
                    pinyin_location += i
                os.system("open -a Safari https://www.google.com/maps/place/" + pinyin_location + "/&amp;")
            
            #打开word，并输入文字
            elif "编辑" in data:
                text = data[2:]
                speak("走你")
                os.system("python3 ./docx_commands/create_doc.py -t "+text)
                os.system("open demo.docx")
            # 调整字体大小
            elif "放大" in data:
                speak("行嘞")
                os.system("python3 ./docx_commands/larger_font.py")
                os.system("open demo.docx")
            # 加粗字体
            elif "加粗" in data:
                speak("好的")
                os.system("python3 ./docx_commands/bold_text.py")
                os.system("open demo.docx")
            # 罗氏理解万岁
            elif "蠢" in data:
                speak("理解万岁")
            # 挥一挥手
            elif "再见" in data:
                speak("再见")
                break


# 跑起
jarvis()

你说：小辣椒
我在！
你说：你太蠢了
理解万岁
你说：我是被这句话来解释一下什么呀
无法从谷歌服务中找到符合你要求的内容; recognition connection failed: [Errno 32] Broken pipe
无法从谷歌服务中找到符合你要求的内容; recognition request failed: Bad Gateway


不仅仅是语音前端。

包括应用场景：微信，slack，Facebook Messager，等等 都可以把我们的ChatBot给integrate进去。

大家有兴趣可以报名我们的课程

一点点来学起机器学习与人工智能

欧了~